In [171]:
import pandas as pd
import numpy as np
import datetime as dt
import itertools
import warnings

warnings.filterwarnings('ignore')

In [ ]:
def claims_formatter(doc):
    doc_copy = doc.copy()
    doc_copy.index = doc_copy.AUTOMOTORES
    formatted_doc = doc_copy.iloc[:,4:].stack(level = [0,1,2,3]).reset_index([1,2,3,4]).drop_duplicates()

    formatted_doc.columns = ['Tipo', 'Periodo', 'Desarrollo 1', 'Fecha Valuacion', 'Pendientes', 'Pagados', 'Incurridos', 'Cantidad']
    formatted_doc['Fecha Ocurrencia'] = int(formatted_doc['Periodo'][1][0:5])
    formatted_doc['Fecha Valuacion'] = [int(i[-4:]) for i in formatted_doc['Fecha Valuacion']]
    formatted_doc['Desarrollo'] = formatted_doc['Fecha Valuacion'] - formatted_doc['Fecha Ocurrencia']
    formatted_doc.drop(['Tipo', 'Periodo', 'Desarrollo 1'], axis = 1, inplace = True)
    return formatted_doc


In [173]:
stos_doc = pd.read_excel(r'D:\Documentos\Python_script\Github\IBNR-NIIF-17\sample\ssn_20232024_desarrollo_siniestros_automotores.xlsx', sheet_name = None, header=[0,1,2,3,4,5])
stos_doc = dict(itertools.islice(stos_doc.items(), 3, len(stos_doc)))

lst = []
for k in stos_doc.keys():
    formated_doc = claims_formatter(stos_doc[k])
    lst.append(formated_doc)

db_stos = pd.concat(lst)

In [174]:
db_stos

,Fecha Valuacion,Pendientes,Pagados,Incurridos,Cantidad,Fecha Ocurrencia,Desarrollo
"([HG], Total, nan, nan)",2024,5.746226e+11,1.170350e+12,1.744972e+12,3736027.0,2023,1
"([HG], RC lesiones 3º transportados, 1, nan)",2024,5.208229e+09,1.942691e+09,7.150920e+09,24934.0,2023,1
"([HG], RC lesiones 3º no transportados, 2, nan)",2024,6.539982e+10,5.208872e+10,1.174885e+11,193752.0,2023,1
"([HG], RC daños a cosas, 3, nan)",2024,1.586698e+11,1.996540e+11,3.583238e+11,1682952.0,2023,1
"([HG], Robo parcial, 4, nan)",2024,1.227506e+10,1.157018e+11,1.279768e+11,384975.0,2023,1
...,...,...,...,...,...,...,...
"([TV x HG], nan, M3, 9.0)",2020,5.000000e+01,2.219781e+06,2.219831e+06,8.0,2018,2
"([TV x HG], nan, M3, 9.0)",2021,5.000000e+01,2.494209e+06,2.494259e+06,9.0,2018,3
"([TV x HG], nan, M3, 9.0)",2022,5.000000e+01,2.494209e+06,2.494259e+06,9.0,2018,4
"([TV x HG], nan, M3, 9.0)",2023,5.000000e+01,2.494209e+06,2.494259e+06,9.0,2018,5
